In [9]:

import matplotlib.pyplot as plt
import numpy as np
# To solve for cross-section limit
from scipy.optimize import brentq

In [12]:



########################################################################################
# Calculate the sensitivity/expected sensitivity
########################################################################################
 
# Number of target Argon nuclei and livetime of DUNE
NA_dune = 4 * 1.5e32             # 40 kton
livetime_dune = 10.0 * 3.154e7   # 10 years

sigma2 = 2**2 # 2 sigma -> 95% C.L.
# Given
gZp = 1.0  # always 1 in event generation
MB  = np.array([5., 10., 20., 40.])   # lighter DM mass
gam = np.array([1.1, 1.5, 10.])       # gamma

# All combinations using broadcasting
MB_grid  = MB[None, :]        # shape (1, 4)
gam_grid = gam[:, None]       # shape (3, 1)

MA_grid  = gam_grid * MB_grid # shape (3, 4) = all combos of gam*MB


print(MA_grid.flatten())
#Flux and BDM-Ar are obtained from the phemenological paper, https://arxiv.org/pdf/1912.05558.pdf
#  flux
flux = np.array([
    634.1,
    303.6,
    117.7,
    36.38,
    468.3,
    203.4,
    72.48,
    19.10,
    28.12,
    7.521,
    2.455,
    0.431
])
#  argon cross section
sigAr = np.array([
    9.057e-30,
    1.063e-29,
    1.220e-29,
    1.278e-29,
    4.978e-29,
    5.609e-29,
    5.965e-29,
    6.152e-29,
    1.270e-27,
    1.377e-27,
    1.437e-27,
    1.470e-27
])

###############################################################
#     PAPER --- EFFICIENCY AND BACKGROUND EVENTS              #
###############################################################

# Nbr. of expected events in nu experiments at g_Z' = 1:
# Number of target nuclei * livetime * flux * cross-section on Argon * efficiency

#  DUNE efficiency
effDUNE_paper = np.array([
    0.4917,
    0.4788,
    0.4973,
    0.5027,
    0.6532,
    0.6660,
    0.6752,
    0.6694,
    0.7635,
    0.7673,
    0.8366,
    0.8512
])

 # expected bkg events after cuts at DUNE
bkgDUNE_paper = np.array([
    10006,
    10006,
    10634,
    11300,
    11894,
    11894,
    11894,
    11894,
    3723,
    3075,
    3075,
    3075
])

evtsDUNE_paper = NA_dune * livetime_dune * flux * sigAr * effDUNE_paper

###############################################################
###############################################################


# Redefining variables for the Paper. Data from Table III, https://arxiv.org/pdf/1912.05558.pdf

evtsLimDUNE = []
for i in range(len(MA_grid.flatten())):
    def sigmasq(s):
        return 2.0 * ((s + bkgDUNE_paper[i]) * np.log(1.0 + s / bkgDUNE_paper[i]) - s) - sigma2
        # Solve for the number of signal events required for sensitivity
        # We need to give a search range: take sqrt(N_b)/10 to sqrt(N_b) * 10
        # For any "reasonable" number of background events this should be enough

    evtsLimDUNE.append(brentq(sigmasq, np.sqrt(
        bkgDUNE_paper[i])/10.0, np.sqrt(bkgDUNE_paper[i] * 10.0)))

PaperDLv = np.sqrt(evtsLimDUNE / evtsDUNE_paper)

print(evtsDUNE_paper)

print(PaperDLv)

[  5.5  11.   22.   44.    7.5  15.   30.   60.   50.  100.  200.  400. ]
[5.34386242e+14 2.92416660e+14 1.35134952e+14 4.42298405e+13
 2.88162966e+15 1.43788303e+15 5.52425867e+14 1.48849829e+14
 5.15989683e+15 1.50379164e+15 5.58520431e+14 1.02056145e+14]
[6.12877527e-07 8.28514808e-07 1.23738536e-06 2.19586539e-06
 2.75543123e-07 3.90073889e-07 6.29320177e-07 1.21236806e-06
 1.54204697e-07 2.72382985e-07 4.46945198e-07 1.04557268e-06]
